In [1]:
%reload_ext autoreload
%autoreload 2

# Imports
Import the required classes and functions.

In [2]:
from mleko.dataset.convert import CSVToVaexConverter
from mleko.dataset.ingest import KaggleIngester
from mleko.dataset.feature_select import CompositeFeatureSelector, MissingRateFeatureSelector, PearsonCorrelationFeatureSelector
from mleko.dataset.split import RandomSplitter
from mleko.dataset.transform import LabelEncoderTransformer, CompositeTransformer, MinMaxScalerTransformer
from mleko.pipeline import Pipeline
from mleko.pipeline.steps import ConvertStep, IngestStep, SplitStep, TransformStep, FeatureSelectStep

from mleko.utils.custom_logger import CustomLogger
import logging

CustomLogger.set_global_log_level(logging.DEBUG)


# Constants
Define configuration variables.

In [3]:
# Kaggle dataset identifier
OWNER_SLUG = "jsphyg"
DATASET_SLUG = "weather-dataset-rattle-package"
DATASET_NAME = f"{OWNER_SLUG}/{DATASET_SLUG}"

# Dataset feature configuration
TARGET_FEATURE = "RainTomorrow"
DATE_FEATURE = "Date"
META_FEATURES = [TARGET_FEATURE, DATE_FEATURE]

# General Configuration
RANDOM_STATE = 1337


# Pipeline Setup

In [4]:
# Download the dataset from Kaggle
from calendar import c


kaggle_data_source = KaggleIngester(
    destination_directory=f"data/{DATASET_NAME}/raw", owner_slug=OWNER_SLUG, dataset_slug=DATASET_SLUG
)

# Convert the CSV file to Apache Arrow format
csv_to_arrow_converter = CSVToVaexConverter(
    output_directory=f"data/{DATASET_NAME}/converted",
    na_values=["NA"],
    true_values=["Yes"],
    false_values=["No"],
    downcast_float=True,
    random_state=RANDOM_STATE,
)

# Divide the dataset into train and test sets
random_data_splitter = RandomSplitter(
    cache_directory=f"data/{DATASET_NAME}/split",
    data_split=(0.80, 0.20),
    shuffle=True,
    stratify=TARGET_FEATURE,
    random_state=RANDOM_STATE,
)

# Configure all the feature selectors in a grouped composite feature selector
# The feature selectors are applied in the order they are defined and can be chained as needed
composite_feature_selector = CompositeFeatureSelector(
    cache_directory=f"data/{DATASET_NAME}/feature_select",
    feature_selectors=[
        MissingRateFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_select",
            missing_rate_threshold=0.5,
            ignore_features=META_FEATURES,
        ),
        PearsonCorrelationFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_select",
            correlation_threshold=0.6,
            ignore_features=META_FEATURES,
        ),
    ],
)

# Configure all the transformers in a grouped composite transformer
# The transformers are applied in the order they are defined and can be chained as needed
composite_transformer = CompositeTransformer(
    cache_directory=f"data/{DATASET_NAME}/transform",
    transformers=[
        LabelEncoderTransformer(
            cache_directory=f"data/{DATASET_NAME}/transform", features=["WindGustDir", "WindDir9am", "WindDir3pm"]
        ),
        MinMaxScalerTransformer(
            cache_directory=f"data/{DATASET_NAME}/transform", features=["WindGustSpeed"], min_value=1, max_value=2
        ),
    ],
)

# Configure the pipeline and the data flow
pipeline = Pipeline(
    steps=[
        IngestStep(kaggle_data_source, outputs=["raw_csv"]),
        ConvertStep(csv_to_arrow_converter, inputs=["raw_csv"], outputs=["df_clean"]),
        SplitStep(random_data_splitter, inputs=["df_clean"], outputs=["df_clean_train_validate", "df_clean_test"]),
        FeatureSelectStep(
            composite_feature_selector,
            fit=True,
            inputs=["df_clean_train_validate"],
            outputs=["df_selected_train_validate"],
            cache_group="fit_transform_train_validate",
        ),
        FeatureSelectStep(
            composite_feature_selector,
            fit=False,
            inputs=["df_clean_test"],
            outputs=["df_selected_test"],
            cache_group="transform_test",
        ),
        TransformStep(
            composite_transformer,
            fit=True,
            inputs=["df_selected_train_validate"],
            outputs=["df_transformed_train_validate"],
            cache_group="fit_transform_train_validate",
        ),
        TransformStep(
            composite_transformer,
            fit=False,
            inputs=["df_selected_test"],
            outputs=["df_transformed_test"],
            cache_group="fit_transform_test",
        ),
    ]
)

[2023-07-08 00:26:20] [INFO] Attempting to fetch Kaggle API credentials from environment variables 'KAGGLE_USERNAME' and 'KAGGLE_KEY'. (kaggle_ingester.py:74)
[2023-07-08 00:26:20] [INFO] Kaggle API credentials not found in environment variables, attempting to fetch from fallback path at ~/.kaggle/kaggle.json. (kaggle_ingester.py:82)
[2023-07-08 00:26:20] [INFO] Kaggle credentials successfully fetched. (kaggle_ingester.py:91)


# Run Pipeline

In [5]:
data_container =  pipeline.run().data

[2023-07-08 00:26:22] [INFO] No data container provided. Creating an empty one. (pipeline.py:77)
[2023-07-08 00:26:22] [INFO] Executing step 1/7: IngestStep. (pipeline.py:81)
[2023-07-08 00:26:23] [INFO] Cache Miss: Downloading jsphyg/weather-dataset-rattle-package/* to data/jsphyg/weather-dataset-rattle-package/raw from Kaggle. (kaggle_ingester.py:292)


[2023-07-08 00:26:24] [INFO] Finished downloading 1 files from Kaggle. (kaggle_ingester.py:303)
[2023-07-08 00:26:24] [INFO] Finished step 1/7 execution. (pipeline.py:83)
[2023-07-08 00:26:24] [INFO] Executing step 2/7: ConvertStep. (pipeline.py:81)
[2023-07-08 00:26:25] [INFO] Cache Miss (LRUCache) CSVToVaexConverter.convert: Executing method. (cache_mixin.py:162)


Converting CSV files:   0%|          | 0/1 [00:00<?, ?it/s]

Writing DataFrame to .arrow file:   0%|          | 0/100 [00:00<?, ?it/s]

[2023-07-08 00:26:28] [INFO] Finished step 2/7 execution. (pipeline.py:83)
[2023-07-08 00:26:28] [INFO] Executing step 3/7: SplitStep. (pipeline.py:81)
[2023-07-08 00:26:28] [INFO] Cache Miss (LRUCache) RandomSplitter.split: Executing method. (cache_mixin.py:162)
[2023-07-08 00:26:28] [INFO] Shuffling data before splitting. (random_splitter.py:135)
[2023-07-08 00:26:28] [INFO] Splitting data with stratification on column 'RainTomorrow'. (random_splitter.py:139)
[2023-07-08 00:26:28] [INFO] Split dataframe into two dataframes with shapes (116368, 24) and (29092, 24). (random_splitter.py:151)


...

[2023-07-08 00:26:29] [INFO] Finished step 5/7 execution. (pipeline.py:83)
[2023-07-08 00:26:29] [INFO] Executing step 6/7: TransformStep. (pipeline.py:81)
[2023-07-08 00:26:29] [INFO] Cache Miss (LRUCache) CompositeTransformer.transform: Executing method. (cache_mixin.py:162)
[2023-07-08 00:26:29] [INFO] Executing composite feature transformation step 1/2: LabelEncoderTransformer. (composite_transformer.py:126)
[2023-07-08 00:26:29] [INFO] Fitting label encoder transformer (3): ('WindGustDir', 'WindDir9am', 'WindDir3pm'). (label_encoder_transformer.py:93)
[2023-07-08 00:26:29] [INFO] Transforming features using label encoding (3): ('WindGustDir', 'WindDir9am', 'WindDir3pm'). (label_encoder_transformer.py:83)
[2023-07-08 00:26:29] [INFO] Finished composite transformation step 1/2. (composite_transformer.py:132)
[2023-07-08 00:26:29] [INFO] Executing composite feature transformation step 2/2: MinMaxScalerTransformer. (composite_transformer.py:126)
[2023-07-08 00:26:29] [INFO] Fitting mi

Writing DataFrame to .arrow file:   0%|          | 0/100 [00:00<?, ?it/s]

[2023-07-08 00:26:29] [INFO] Saving transformer to data/jsphyg/weather-dataset-rattle-package/transform/CompositeTransformer.transform.fit_transform_train_validate.5f7ac699de5bfd3305c122756f83951a.transformer. (base_transformer.py:128)
[2023-07-08 00:26:29] [INFO] Finished step 6/7 execution. (pipeline.py:83)
[2023-07-08 00:26:29] [INFO] Executing step 7/7: TransformStep. (pipeline.py:81)
[2023-07-08 00:26:29] [INFO] Cache Miss (LRUCache) CompositeTransformer.transform: Executing method. (cache_mixin.py:162)
[2023-07-08 00:26:29] [INFO] Executing composite feature transformation step 1/2: LabelEncoderTransformer. (composite_transformer.py:126)
[2023-07-08 00:26:29] [INFO] Transforming features using label encoding (3): ('WindGustDir', 'WindDir9am', 'WindDir3pm'). (label_encoder_transformer.py:83)
[2023-07-08 00:26:29] [INFO] Finished composite transformation step 1/2. (composite_transformer.py:132)
[2023-07-08 00:26:29] [INFO] Executing composite feature transformation step 2/2: MinMax

Writing DataFrame to .arrow file:   0%|          | 0/100 [00:00<?, ?it/s]

[2023-07-08 00:26:30] [INFO] Finished step 7/7 execution. (pipeline.py:83)


In [6]:
data_container['df_transformed_test']

#,Date,Location,Rainfall,Evaporation,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,Humidity3pm,Cloud3pm,RainToday,RainTomorrow
0,2008-12-09 00:00:00,Albury,0.0,--,16,1.5736434108527133,4,7,9,--,False,True
1,2008-12-15 00:00:00,Albury,0.0,--,0,--,8,10,32,--,False,--
2,2008-12-20 00:00:00,Albury,0.0,--,13,1.1550387596899225,4,16,26,--,False,False
3,2008-12-24 00:00:00,Albury,0.0,--,3,1.2093023255813953,1,7,23,--,False,False
4,2008-12-27 00:00:00,Albury,0.0,--,12,1.3953488372093024,0,11,28,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
"29,087",2017-05-28 00:00:00,Uluru,0.0,--,6,1.2093023255813953,4,1,25,--,False,False
"29,088",2017-05-31 00:00:00,Uluru,0.0,--,6,1.310077519379845,6,6,32,--,False,False
"29,089",2017-06-04 00:00:00,Uluru,0.0,--,6,1.193798449612403,1,6,26,--,False,False
"29,090",2017-06-05 00:00:00,Uluru,0.0,--,6,1.2403100775193798,1,1,11,--,False,False
